In [7]:
import pandas as pd 
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#df = tuxml.load_dataset()
#path = f"x264/sampledConfigurations_{name_of_technique}_t3.csv"
#lst = ['distBased', 'divDistBased', 'henard', 'random', 'solverBased', 'twise']

path = r"sampledConfigurations_divDistBased_t3.csv"


df = pd.read_csv(path, sep=';')


In [8]:
df

,root,polly_vectorizer,none,polly,polly_parallel,parallel,parallel_force,polly_no_tiling,polly_delinearize,polly_dependences_analysis_type,...,pollyoptmaxcoefficient_100,pollyoptmaxcoefficient_1000,pollyoptmaxcoefficient_10000,pollydefaulttilesize,pollydefaulttilesize_4,pollydefaulttilesize_16,pollydefaulttilesize_64,pollydefaulttilesize_256,pollydefaulttilesize_1024,Performance
0,1,1,0,1,1,0,1,1,1,1,...,0,0,0,1,0,0,0,0,1,23.980000
1,1,1,0,1,1,0,1,1,1,1,...,0,0,0,1,0,0,1,0,0,24.000000
2,1,1,0,1,1,0,1,0,1,1,...,0,0,1,1,0,0,0,1,0,6.415000
3,1,1,0,1,1,0,1,0,1,1,...,0,0,1,1,0,0,1,0,0,4.245000
4,1,1,0,1,0,0,0,0,1,1,...,0,1,0,1,0,1,0,0,0,29.703333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2167,1,1,1,0,0,0,0,0,1,1,...,1,0,0,1,0,0,1,0,0,29.116667
2168,1,1,0,1,0,0,0,0,1,1,...,0,0,0,1,0,0,0,1,0,28.573333
2169,1,1,1,0,0,0,0,0,1,1,...,0,1,0,1,0,0,1,0,0,29.076667
2170,1,1,1,0,0,0,0,1,1,1,...,0,0,0,1,1,0,0,0,0,28.563333


### Geração de feature importance sem a utilização de hiperparâmetros

In [9]:
def run_featureSelection(df, hyperparameters=None, path=None):
    X = df.drop('Performance', axis=1)
    y = df['Performance']

    if hyperparameters is None:
        rf = RandomForestRegressor()
    else:
        rf = RandomForestRegressor(**hyperparameters)
    
    rf.fit(X, y)

    importances = rf.feature_importances_

    feature_importances = pd.DataFrame(importances, index = X.columns, columns=['importance']).sort_values('importance', ascending=False)

    print(feature_importances)
    
    if hyperparameters is None:
        if path is None:
            feature_importances.to_csv("feature_importance.csv", header=True)
        else:
            feature_importances.to_csv("feature_importance_"+path+".csv", header=True)
    else:
        if path is None:
            feature_importances.to_csv("feature_importanceHP.csv", header=True)
        else:
            feature_importances.to_csv("feature_importanceHP_"+path+".csv", header=True)


run_featureSelection(df)

                                 importance
max                                0.494175
polly_no_tiling                    0.206428
min                                0.110778
value_based                        0.061392
memory_based                       0.059193
parallel                           0.022040
none                               0.020009
polly                              0.011969
pollydefaulttilesize_4             0.006546
pollydefaulttilesize_1024          0.001472
pollydefaulttilesize_64            0.001251
polly_parallel                     0.000544
pollydefaulttilesize_256           0.000476
pollydefaulttilesize_16            0.000436
polly_opt_simplify_deps            0.000262
pollyoptmaxconstantterm_10000      0.000231
pollyrtcmaxparameters_2            0.000228
pollyoptmaxconstantterm_1          0.000225
pollyoptmaxcoefficient_10000       0.000203
pollyoptmaxconstantterm_10         0.000197
pollyrtcmaxparameters_1            0.000194
pollyoptmaxconstantterm_1000    

### Geração de feature importance com a utilização de hiperparâmetros

In [10]:
hyperparams_RF = {
    "max_depth": 25,
    "n_estimators": 10,
    "n_jobs" : -1
}

run_featureSelection(df, hyperparams_RF)

                                 importance
max                                0.579334
polly_no_tiling                    0.151858
memory_based                       0.107284
value_based                        0.094610
parallel                           0.023035
none                               0.015006
polly                              0.010729
pollydefaulttilesize_4             0.006075
min                                0.004766
pollydefaulttilesize_1024          0.001356
pollydefaulttilesize_64            0.001186
polly_parallel                     0.000699
pollydefaulttilesize_256           0.000476
pollydefaulttilesize_16            0.000390
polly_opt_simplify_deps            0.000307
pollyoptmaxconstantterm_1          0.000245
pollyoptmaxconstantterm_10000      0.000214
pollyoptmaxconstantterm_10         0.000208
pollyrtcmaxparameters_2            0.000201
pollyoptmaxcoefficient_10000       0.000198
pollyrtcmaxparameters_1            0.000190
pollyrtcmaxparameters_4         

In [12]:
samples_config = ['distBased', 'divDistBased','henard', 'random', 'solverBased', 'twise']

for config_name in samples_config:
    for t in [1,2,3]:
        path = r"sampledConfigurations_"+config_name+"_t"+str(t)+".csv"
        path_name = config_name+"_t"+str(t)
        df = pd.read_csv(path, sep=';')
        
        run_featureSelection(df, None, path_name)
        run_featureSelection(df, hyperparams_RF, path_name)

                                 importance
max                                0.339765
polly_no_tiling                    0.237768
min                                0.191002
pollyrtcmaxparameters_8            0.037513
pollydefaulttilesize_1024          0.028150
pollyoptmaxconstantterm_1          0.018731
pollydefaulttilesize_4             0.017756
pollyrtcmaxparameters_4            0.017479
polly                              0.016800
parallel_force                     0.014418
pollyoptmaxconstantterm_100        0.012012
parallel                           0.011984
polly_opt_simplify_deps            0.010256
none                               0.010131
pollydefaulttilesize_16            0.009261
pollydefaulttilesize_64            0.006434
pollyoptmaxcoefficient_10000       0.005866
polly_parallel                     0.004874
pollyrtcmaxparameters_1            0.004061
pollyoptmaxcoefficient_1           0.003825
value_based                        0.001746
pollyoptmaxconstantterm_10000   

                                 importance
polly_no_tiling                    0.283575
max                                0.244160
memory_based                       0.162650
min                                0.155672
value_based                        0.067201
parallel                           0.029749
pollydefaulttilesize_4             0.021994
polly                              0.016734
none                               0.013534
pollydefaulttilesize_64            0.001165
pollydefaulttilesize_1024          0.000830
pollydefaulttilesize_256           0.000579
pollydefaulttilesize_16            0.000306
pollyrtcmaxparameters_1            0.000169
pollyoptmaxcoefficient_10          0.000156
pollyoptmaxconstantterm_1          0.000140
pollyoptmaxcoefficient_10000       0.000137
pollyoptmaxcoefficient_1           0.000130
pollyoptmaxconstantterm_100        0.000115
polly_opt_simplify_deps            0.000111
parallel_force                     0.000107
polly_parallel                  

                                 importance
max                                0.527990
polly_no_tiling                    0.177855
min                                0.079788
value_based                        0.076631
memory_based                       0.070424
parallel                           0.023255
none                               0.017310
polly                              0.012396
pollydefaulttilesize_4             0.006999
pollydefaulttilesize_1024          0.001450
pollydefaulttilesize_64            0.001267
pollydefaulttilesize_256           0.000463
pollydefaulttilesize_16            0.000435
polly_parallel                     0.000403
polly_opt_simplify_deps            0.000277
pollyoptmaxconstantterm_1          0.000234
pollyrtcmaxparameters_2            0.000232
pollyoptmaxconstantterm_10000      0.000214
pollyoptmaxcoefficient_10000       0.000208
pollyoptmaxconstantterm_1000       0.000201
pollyoptmaxcoefficient_10          0.000201
pollyrtcmaxparameters_1         

                                   importance
polly_no_tiling                  2.717208e-01
max                              2.165877e-01
memory_based                     1.625957e-01
value_based                      1.242363e-01
min                              1.164554e-01
parallel                         4.555561e-02
polly                            2.192532e-02
none                             1.970932e-02
polly_parallel                   4.373460e-03
parallel_force                   2.670362e-03
pollyoptmaxconstantterm_1        2.596562e-03
pollydefaulttilesize_4           2.111506e-03
pollyoptmaxcoefficient_100       1.620992e-03
polly_opt_simplify_deps          1.225286e-03
pollyrtcmaxparameters_1          9.576106e-04
pollyrtcmaxparameters_2          8.912066e-04
pollyoptmaxconstantterm_1000     7.622379e-04
pollyrtcmaxparameters_4          7.504755e-04
pollyoptmaxconstantterm_10000    5.819543e-04
pollyrtcmaxparameters_16         5.659750e-04
pollyoptmaxcoefficient_10        5

                                 importance
polly_no_tiling                    0.437463
max                                0.146112
memory_based                       0.132184
value_based                        0.108937
min                                0.093633
parallel                           0.026444
polly                              0.023935
none                               0.014961
pollydefaulttilesize_4             0.005072
pollyrtcmaxparameters_1            0.002158
polly_parallel                     0.001531
pollyoptmaxconstantterm_10000      0.001025
parallel_force                     0.000732
pollydefaulttilesize_64            0.000604
pollydefaulttilesize_1024          0.000598
pollyoptmaxcoefficient_1000        0.000490
polly_opt_simplify_deps            0.000465
pollyoptmaxconstantterm_1          0.000397
pollyoptmaxcoefficient_1           0.000382
pollyrtcmaxparameters_8            0.000339
pollydefaulttilesize_256           0.000315
pollyrtcmaxparameters_16        

                                 importance
memory_based                       0.283046
max                                0.213974
min                                0.139175
polly                              0.137001
value_based                        0.114733
polly_parallel                     0.055345
parallel_force                     0.031238
parallel                           0.018987
polly_opt_simplify_deps            0.003360
polly_no_tiling                    0.002389
none                               0.000751
root                               0.000000
pollyoptmaxcoefficient_100         0.000000
pollyoptmaxcoefficient_1           0.000000
pollyoptmaxcoefficient_10          0.000000
pollyoptmaxcoefficient_10000       0.000000
pollyoptmaxcoefficient             0.000000
pollyoptmaxcoefficient_1000        0.000000
pollydefaulttilesize_16            0.000000
pollydefaulttilesize               0.000000
pollydefaulttilesize_4             0.000000
pollyoptmaxconstantterm_1000    

                                 importance
min                                0.223820
value_based                        0.220586
memory_based                       0.197136
max                                0.164803
pollydefaulttilesize_4             0.083779
pollydefaulttilesize_1024          0.050592
pollyrtcmaxparameters_1            0.014684
none                               0.008891
polly_parallel                     0.007445
pollydefaulttilesize_16            0.007442
pollydefaulttilesize_64            0.005245
polly                              0.005082
pollyoptmaxcoefficient_10000       0.004929
parallel_force                     0.003024
pollyrtcmaxparameters_2            0.001225
pollyrtcmaxparameters_4            0.000543
pollydefaulttilesize_256           0.000364
parallel                           0.000262
pollyoptmaxconstantterm_10000      0.000047
pollyoptmaxconstantterm_1000       0.000035
polly_no_tiling                    0.000016
pollyoptmaxconstantterm_100     

                                 importance
polly_no_tiling                    0.225326
min                                0.218663
memory_based                       0.215144
max                                0.145611
value_based                        0.143905
pollydefaulttilesize_4             0.016988
none                               0.009655
polly                              0.008964
parallel                           0.007873
polly_parallel                     0.002658
pollydefaulttilesize_1024          0.001511
pollydefaulttilesize_64            0.001252
pollydefaulttilesize_16            0.000425
pollydefaulttilesize_256           0.000247
pollyoptmaxconstantterm_10000      0.000152
polly_opt_simplify_deps            0.000136
pollyoptmaxcoefficient_10000       0.000122
pollyoptmaxcoefficient_1           0.000118
pollyoptmaxcoefficient_100         0.000115
pollyoptmaxconstantterm_1          0.000115
pollyrtcmaxparameters_1            0.000111
pollyoptmaxconstantterm_1000    